I decided to write the notebook from the scratch, since the I did not like the template we had

In [2]:
#downloading the dataset
import os
path_do_data = '../../datasets/Machine_translation_EN_RU/data.txt'
if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc
    path_do_data = './data.txt'

Dataset not found locally. Downloading from github.
File ‘data.txt’ already there; not retrieving.



In [3]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [86]:
import torch
import transformers
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric, Dataset, DatasetDict

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [77]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
metric = load_metric("sacrebleu")


In [65]:
data_pandas = pd.read_csv('data.txt', delimiter='\t', header=None)
data_pandas.columns = ['en', 'ru']
data_pandas


#80, 15, 5 split
train_df, test_val_df = train_test_split(data_pandas, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.25, random_state=42)

# Convert the dataframes to dictionaries
train_dict = {'source': train_df['en'].tolist(), 'target': train_df['ru'].tolist()}
val_dict = {'source': val_df['en'].tolist(), 'target': val_df['ru'].tolist()}
test_dict = {'source': test_df['en'].tolist(), 'target': test_df['ru'].tolist()}

# Create the Hugging Face dataset
train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

In [70]:
train_dataset[:5]

{'source': ['In Liverpool’s business district, Days Inn Liverpool is a 10-minute walk from the famous Albert Dock.',
  'The nearest airport is Netaji Subhash Chandra Bose International Airport, 16 km from the property.',
  'The self-catering house features a seating area with sofas and TV. There is also a fully equipped kitchen stovetop, refrigerator and kitchenware.',
  'At Genipabu Praia Hotel you will find a 24-hour reception, a garden and a terrace.',
  'Simply furnished, the accommodation will provide you with cable TV and night tables with lamps.'],
 'target': ['Отель Days Inn Liverpool находится в деловом районе Ливерпуля, в 10 минутах ходьбы от знаменитого Альберт-Дока.',
  'Расстояние до ближайшего международного аэропорта имени Нетаджи Субхас Чандра Боса составляет 16 км.',
  'К услугам гостей мягкий уголок с диванами и телевизором и полностью оборудованная собственная кухня с плитой, холодильником и посудой.',
  'В распоряжении гостей отеля Genipabu Praia круглосуточная стой

In [79]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [80]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-ru', vocab_size=62518, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [95]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ru"

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

def preprocess_function(examples):
    inputs = examples['source']
    targets = examples['target']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [96]:
raw_datasets = DatasetDict({
    'train':train_dataset,
    'val': val_dataset,
    'test': test_dataset
})

In [98]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [99]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [110]:
batch_size = 16
model_name = model_checkpoint.split("/")[0]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    #push_to_hub=True,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [107]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [111]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [112]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.876800,0.792554,33.649600,27.845200


TrainOutput(global_step=2500, training_loss=0.9871775756835938, metrics={'train_runtime': 909.2136, 'train_samples_per_second': 43.994, 'train_steps_per_second': 2.75, 'total_flos': 472954224771072.0, 'train_loss': 0.9871775756835938, 'epoch': 1.0})